In [2]:
import os
import random
import shutil

# Path to the folder containing the PNG images
folder_path = '/Users/user/Downloads/pneumonia'

# List all files in the folder, filtering out directories
file_names = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

# Shuffle the list of file names randomly
random.shuffle(file_names)

# Calculate the number of files for training (80%) and testing (20%)
num_train = int(0.8 * len(file_names))
num_test = len(file_names) - num_train

# Create directories for training and testing data
train_dir = '/Users/user/Downloads/pneumonia/train'
test_dir = '/Users/user/Downloads/pneumonia/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Copy 80% of the files to the training directory
for file_name in file_names[:num_train]:
    src_path = os.path.join(folder_path, file_name)
    dst_path = os.path.join(train_dir, file_name)
    shutil.copy(src_path, dst_path)

# Copy 20% of the files to the testing directory
for file_name in file_names[num_train:]:
    src_path = os.path.join(folder_path, file_name)
    dst_path = os.path.join(test_dir, file_name)
    shutil.copy(src_path, dst_path)

print("Data partitioning completed successfully.")


Data partitioning completed successfully.


In [3]:
import cv2
import numpy as np

def add_poisson(img, lambda_val):
    poisson_noise = np.random.poisson(lambda_val, size=img.shape)
    noisy_image = img + poisson_noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

# Function to apply noise to images in a folder
def apply_noise_to_folder(folder_path, output_folder, noise_type, noise_params):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.png'):
            img_path = os.path.join(folder_path, file_name)
            img = cv2.imread(img_path, 0)
            for param in noise_params:
                noisy_img = None
                if noise_type == 'poisson':
                    noisy_img = add_poisson(img.copy(), param)
                output_subfolder = os.path.join(output_folder, str(param))
                os.makedirs(output_subfolder, exist_ok=True)
                output_path = os.path.join(output_subfolder, file_name)
                cv2.imwrite(output_path, noisy_img)

# Path to the training and test folders
train_folder = '/Users/user/Downloads/pneumonia/train'
test_folder = '/Users/user/Downloads/pneumonia/test'

# Output folders for noisy training and test sets
noisy_train_folder = '/Users/user/Downloads/pneumonia/noisy_train'
noisy_test_folder = '/Users/user/Downloads/pneumonia/noisy_test'

# Lambda values for Poisson noise
lambda_values = [25, 50, 75]

# Apply noise to training and test sets
apply_noise_to_folder(train_folder, noisy_train_folder, 'poisson', lambda_values)
apply_noise_to_folder(test_folder, noisy_test_folder, 'poisson', lambda_values)

print("Noise applied to training and test sets.")

Noise applied to training and test sets.


In [7]:
import cv2
import numpy as np
import os

# Define paths to folders
train_folder = '/Users/user/Downloads/pneumonia/train'
test_folder = '/Users/user/Downloads/pneumonia/test'
noisy_train_folder = '/Users/user/Downloads/pneumonia/noisy_train/25'
noisy_test_folder = '/Users/user/Downloads/pneumonia/noisy_test/25'

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images

# Load clean and noisy images for training and testing
def load_data(train_folder, noisy_train_folder, test_folder, noisy_test_folder):
    clean_train_images = load_images_from_folder(train_folder)
    clean_test_images = load_images_from_folder(test_folder)
    noisy_train_images = load_images_from_folder(noisy_train_folder)
    noisy_test_images = load_images_from_folder(noisy_test_folder)
    return clean_train_images, clean_test_images, noisy_train_images, noisy_test_images

# Normalize the images
def normalize_images(images):
    normalized_images = []
    for image in images:
        if image.shape[-1] == 1:
            normalized_image = np.squeeze(image, axis=-1)  # Remove singleton dimension
        else:
            normalized_image = image.astype('float32') / 255.
        normalized_images.append(normalized_image)
    return normalized_images

# Load data
clean_train_images, clean_test_images, noisy_train_images, noisy_test_images = load_data(train_folder, noisy_train_folder, test_folder, noisy_test_folder)

# Normalize images
clean_train_images = normalize_images(clean_train_images)
clean_test_images = normalize_images(clean_test_images)
noisy_train_images = normalize_images(noisy_train_images)
noisy_test_images = normalize_images(noisy_test_images)

# Function to apply median filter
def apply_median_filter(image, kernel_size):
    return cv2.medianBlur(image, kernel_size)

# Function to apply mean filter
def apply_mean_filter(image, kernel_size):
    return cv2.blur(image, (kernel_size, kernel_size))

# Function to apply bilateral filter
def apply_bilateral_filter(image, d, sigma_color, sigma_space):
    return cv2.bilateralFilter(image, d, sigma_color, sigma_space)

# Function to apply Gaussian filter
def apply_gaussian_filter(image, kernel_size):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

# Step 2: Apply denoising methods to noisy test images for each lambda value
def denoise_images(noisy_images, method, kernel_size=None, d=None, sigma_color=None, sigma_space=None):
    denoised_images = []
    for img in noisy_images:
        if method == 'median':
            denoised_img = apply_median_filter(img, kernel_size)
        elif method == 'mean':
            denoised_img = apply_mean_filter(img, kernel_size)
        elif method == 'bilateral':
            denoised_img = apply_bilateral_filter(img, d, sigma_color, sigma_space)
        elif method == 'gaussian':
            denoised_img = apply_gaussian_filter(img, kernel_size)
        
        # Resize the denoised image to match the dimensions of the clean image
        denoised_img_resized = cv2.resize(denoised_img, (clean_test_images[0].shape[1], clean_test_images[0].shape[0]))
        
        denoised_images.append(denoised_img_resized)
    return denoised_images

# Step 3: Calculate error metric for classical methods
def calculate_error_for_classical_method(denoised_images, clean_test_images):
    errors = []
    for denoised_img, clean_img in zip(denoised_images, clean_test_images):
        # Resize denoised_img to match the shape of clean_img
        denoised_img_resized = cv2.resize(denoised_img, (clean_img.shape[1], clean_img.shape[0]))

        # Calculate error
        error = np.mean(np.square(clean_img - denoised_img_resized))
        errors.append(error)
    return errors


# Step 4: Calculate error metric for CNN autoencoder (already implemented)

# Step 5: Compare error metrics for different filters and lambda values
lambda_values = [25, 50, 75]

for lambda_val in lambda_values:
    print(f"\nLambda = {lambda_val}")

    # Denoise with Median Filter
    if noisy_test_images:
        denoised_images_median = denoise_images(noisy_test_images, 'median', kernel_size=3)
        errors_median = calculate_error_for_classical_method(denoised_images_median, clean_test_images)
        print(f"Average error for Median Filter: {np.mean(errors_median)}")

    # Denoise with Mean Filter
    denoised_images_mean = denoise_images(noisy_test_images, 'mean', kernel_size=3)
    errors_mean = calculate_error_for_classical_method(denoised_images_mean, clean_test_images)
    print(f"Average error for Mean Filter: {np.mean(errors_mean)}")

    # Denoise with Bilateral Filter
    denoised_images_bilateral = denoise_images(noisy_test_images, 'bilateral', d=9, sigma_color=75, sigma_space=75)
    errors_bilateral = calculate_error_for_classical_method(denoised_images_bilateral, clean_test_images)
    print(f"Average error for Bilateral Filter: {np.mean(errors_bilateral)}")

    # Denoise with Gaussian Filter
    denoised_images_gaussian = denoise_images(noisy_test_images, 'gaussian', kernel_size=3)
    errors_gaussian = calculate_error_for_classical_method(denoised_images_gaussian, clean_test_images)
    print(f"Average error for Gaussian Filter: {np.mean(errors_gaussian)}")

def calculate_psnr_for_classical_method(denoised_images, clean_test_images):
    psnrs = []
    for denoised_img, clean_img in zip(denoised_images, clean_test_images):
        # Resize the denoised image to match the dimensions of the clean image
        denoised_img_resized = cv2.resize(denoised_img, (clean_img.shape[1], clean_img.shape[0]))

        mse = np.mean(np.square(clean_img - denoised_img_resized))
        if mse == 0:
            psnr = 100  # PSNR is infinite when MSE is zero
        else:
            max_pixel = 1.0  # Assuming pixel values are in the range [0, 1]
            psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        psnrs.append(psnr)
    return psnrs


# Define lambda values
lambda_values = [25, 50, 75]

# Iterate over lambda values
for lambda_val in lambda_values:
    print(f"\nLambda = {lambda_val}")
    
    # Denoise images using different filters
    denoised_images_median = denoise_images(noisy_test_images, 'median', kernel_size=3)
    denoised_images_mean = denoise_images(noisy_test_images, 'mean', kernel_size=3)
    denoised_images_bilateral = denoise_images(noisy_test_images, 'bilateral', d=9, sigma_color=75, sigma_space=75)
    denoised_images_gaussian = denoise_images(noisy_test_images, 'gaussian', kernel_size=3)
    
    # Calculate PSNR for each method
    psnrs_median = calculate_psnr_for_classical_method(denoised_images_median, clean_test_images)
    psnrs_mean = calculate_psnr_for_classical_method(denoised_images_mean, clean_test_images)
    psnrs_bilateral = calculate_psnr_for_classical_method(denoised_images_bilateral, clean_test_images)
    psnrs_gaussian = calculate_psnr_for_classical_method(denoised_images_gaussian, clean_test_images)
    
    # Print and compare PSNRs
    print(f"Average PSNR for Median Filter (lambda = {lambda_val}): {np.mean(psnrs_median)} dB")
    print(f"Average PSNR for Mean Filter (lambda = {lambda_val}): {np.mean(psnrs_mean)} dB")
    print(f"Average PSNR for Bilateral Filter (lambda = {lambda_val}): {np.mean(psnrs_bilateral)} dB")
    print(f"Average PSNR for Gaussian Filter (lambda = {lambda_val}): {np.mean(psnrs_gaussian)} dB")



Lambda = 25
Average error for Median Filter: 0.08965595066547394
Average error for Mean Filter: 0.08960691094398499
Average error for Bilateral Filter: 0.08928577601909637
Average error for Gaussian Filter: 0.0896238386631012

Lambda = 50
Average error for Median Filter: 0.08965595066547394
Average error for Mean Filter: 0.08960691094398499
Average error for Bilateral Filter: 0.08928577601909637
Average error for Gaussian Filter: 0.0896238386631012

Lambda = 75
Average error for Median Filter: 0.08965595066547394
Average error for Mean Filter: 0.08960691094398499
Average error for Bilateral Filter: 0.08928577601909637
Average error for Gaussian Filter: 0.0896238386631012

Lambda = 25
Average PSNR for Median Filter (lambda = 25): 10.901243109658 dB
Average PSNR for Mean Filter (lambda = 25): 10.9046997008307 dB
Average PSNR for Bilateral Filter (lambda = 25): 10.922254519204891 dB
Average PSNR for Gaussian Filter (lambda = 25): 10.90364058155884 dB

Lambda = 50
Average PSNR for Median 